# VAMOS quickstart: user-friendly API

This notebook demonstrates the user-friendly API for quick optimization tasks.
For the full AutoNSGA-II configuration (external archive, BLX-alpha, non-uniform mutation),
we use the `optimize` API because `run_nsgaii` does not expose those knobs.


In [1]:
from vamos import optimize, OptimizeConfig, NSGAIIConfig, ZDT1
import matplotlib.pyplot as plt
import numpy as np


## 1. Minimal Example (ZDT1)

Here is the absolute simplest way to run an optimization: just provide the problem name.

In [2]:
SEED = 42

problem = ZDT1()

result = optimize(
    OptimizeConfig(
        problem=problem,
        algorithm="nsgaii",
        algorithm_config=(
            NSGAIIConfig()
            .pop_size(56)
            .offspring_size(14)
            .crossover("blx_alpha", prob=0.88, alpha=0.94, repair="clip")
            .mutation("non_uniform", prob="0.45/n", perturbation=0.3)
            .selection("tournament", pressure=9)
            .survival("nsga2")
            .repair("round")
            .external_archive(size=56, archive_type="hypervolume")
            .engine("numpy")
            .fixed()
        ),
        termination=("n_eval", 10000),
        seed=SEED,
    )
)


Running NSGAII on zdt1 for 10000 evaluations...


In [3]:
result.summary()

=== VAMOS Quick Result ===
Algorithm: NSGAII
Problem: ZDT1Problem (n_var=30, n_obj=2)
Evaluations: 10000
Result solutions: 100

Objective ranges:
  f1: [0.000001, 0.998910]
  f2: [0.020138, 1.479031]

Hypervolume (auto ref): 1.419497


In [4]:
result.plot()

<Figure size 640x480 with 1 Axes>

<Axes: title={'center': 'NSGAII on ZDT1Problem'}>

In [5]:
# Knee point (balanced trade-off)
knee = result.best("knee")
print("Knee point:")
print(f"  Objectives: {knee['F']}")
print(f"  Index: {knee['index']}")


Knee point:
  Objectives: [0.08316279 0.407137  ]
  Index: 82


In [6]:
# Export to pandas
df = result.to_dataframe()
df.head(3)

f1        f2            x0        x1        x2        x3  \
0  5.826351e-07  1.479031  5.826351e-07  0.007074  0.006754  0.159477   
1  6.335965e-01  0.038162  6.335965e-01  0.034639  0.001633  0.000788   
2  6.029057e-07  1.168878  6.029057e-07  0.001554  0.017848  0.004245   

         x4        x5        x6        x7  ...       x20       x21       x22  \
0  0.007169  0.057344  0.019122  0.013273  ...  0.009774  0.129323  0.052865   
1  0.000009  0.031580  0.003112  0.027005  ...  0.021577  0.004467  0.001718   
2  0.000219  0.059294  0.001704  0.006371  ...  0.034560  0.008630  0.003130   

        x23       x24       x25       x26       x27       x28       x29  
0  0.033621  0.000334  0.000360  0.004630  0.008300  0.008666  0.000636  
1  0.003975  0.032338  0.029810  0.004940  0.003618  0.002598  0.002275  
2  0.000305  0.003216  0.000388  0.007880  0.000610  0.020421  0.000454  

[3 rows x 32 columns]

## 2. Canonical Engine Path (Serializable Config)

This is the canonical path that powers the CLI, StudyRunner, and automation.
The config is a plain dict (YAML/JSON friendly), so it also works for algorithms without quick wrappers.


In [ ]:
# Canonical engine path: build a config, serialize it, then run optimize().
cfg = (
    NSGAIIConfig()
    .pop_size(100)
    .crossover("sbx", prob=0.9, eta=20.0)
    .mutation("pm", prob="1/n", eta=20.0)
    .selection("tournament", pressure=2)
    .survival("nsga2")
    .engine("numpy")
    .fixed()
)

cfg_dict = cfg.to_dict()  # YAML/JSON friendly for CLI or StudyRunner
canonical_result = optimize(
    OptimizeConfig(
        problem=problem,
        algorithm="nsgaii",
        algorithm_config=cfg_dict,
        termination=("n_eval", 1000),
        seed=SEED,
    )
)
canonical_result.summary()


## 3. Advanced Usage

For full control, use the `optimize` API with an explicit NSGA-II configuration.
Below we rerun with the same AutoNSGA-II settings but a shorter budget.


In [7]:
result2 = optimize(
    OptimizeConfig(
        problem=problem,
        algorithm="nsgaii",
        algorithm_config=(
            NSGAIIConfig()
            .pop_size(56)
            .offspring_size(14)
            .crossover("blx_alpha", prob=0.88, alpha=0.94, repair="clip")
            .mutation("non_uniform", prob="0.45/n", perturbation=0.3)
            .selection("tournament", pressure=9)
            .survival("nsga2")
            .repair("round")
            .external_archive(size=56, archive_type="hypervolume")
            .engine("numpy")
            .fixed()
        ),
        termination=("n_eval", 2000),
        seed=SEED
    )
)
result2.summary()


Running NSGAII on zdt1 for 2000 evaluations...
=== VAMOS Quick Result ===
Algorithm: NSGAII
Problem: ZDT1Problem (n_var=30, n_obj=2)
Evaluations: 2000
Result solutions: 100

Objective ranges:
  f1: [0.000783, 1.050518]
  f2: [0.037841, 19.344400]

Hypervolume (auto ref): 19.645579


## 4. Comparing Results

We can easily compare multiple result objects.

In [8]:
from vamos import pareto_filter

plt.figure()
fronts = []
for res in (result, result2):
    F = res.front()
    if F is not None and F.size:
        fronts.append(F)

if fronts:
    F_all = np.vstack(fronts)
    F_plot = pareto_filter(F_all)
    if F_plot is not None and F_plot.size:
        plt.scatter(F_plot[:, 0], F_plot[:, 1], label="Union PF", alpha=0.7)

plt.title("Comparison of NSGA-II Runs")
plt.xlabel("f1")
plt.ylabel("f2")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


<Figure size 640x480 with 1 Axes>

## 5. Solving a Combinatorial Problem (TSP)

VAMOS isn't just for continuous problems. Here is how to solve a Permutation problem like the Travelling Salesman Problem (TSP).
We use the **Order Crossover (OX)** and **Swap Mutation** operators, which are automatically selected for permutation variables.

In [ ]:
from vamos.foundation.problem.tsp import TSPProblem, _default_coordinates
from vamos import optimize, OptimizeConfig, NSGAIIConfig
# Create a small TSP instance
coords = _default_coordinates()
tsp_prob = TSPProblem(coordinates=coords)
print(f"Created TSP with {len(coords)} cities.")

# Run optimization
print("Optimizing TSP Tour...")
tsp_res = optimize(
    OptimizeConfig(
        problem=tsp_prob,
        algorithm="nsgaii",
        algorithm_config=NSGAIIConfig()
            .pop_size(50)
            .engine("numpy")  # Use NumPy engine (Python loops for TSP)
            .fixed(),
        termination=("n_eval", 2000),
        seed=42
    )
)

# Visualize the Best Tour
# Objective 0 is Total Distance
best_idx = np.argmin(tsp_res.F[:, 0])
best_tour = tsp_res.X[best_idx].astype(int)
best_dist = tsp_res.F[best_idx, 0]

print(f"Best Tour Length: {best_dist:.4f}")
print(f"Tour Indices: {best_tour}")

# Plotting the tour
plt.figure(figsize=(6, 6))
# Plot cities
plt.scatter(coords[:, 0], coords[:, 1], c='red', s=100, zorder=2)
for i, (x, y) in enumerate(coords):
    plt.text(x, y+0.05, str(i), fontsize=12, ha='center')

# Plot edges
tour_coords = coords[best_tour]
# Close the loop
tour_coords = np.vstack([tour_coords, tour_coords[0]])
plt.plot(tour_coords[:, 0], tour_coords[:, 1], 'b-', linewidth=2, zorder=1)

plt.title(f"Best TSP Tour (Length={best_dist:.2f})")
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()
